# Writing a Public DID to the Sovrin StagingNet

### This notebook walks through how to connect an Aca-Py agent to a live Indy network. In our example we will be connecting to the Sovrin Stagingnet. To view details about Sovrin Stagingnet transactions please visit [Indyscan](https://indyscan.io/home/SOVRIN_STAGINGNET).

## 1. Initialise a controller for Issuer Agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://issuer-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Get current public DID

Before being able to write to any indy based ledger, your agent must have a public DID written on the ledger giving it the authority to write to it. As the cell below shows, this agent does not currently have a public DID. So any writes to the ledger will be rejected.

In [2]:
response = await agent_controller.wallet.get_public_did()
print(response)

{'result': {'did': 'MjFQDoitt2us2JNpLBGnSY', 'verkey': 'CJGSRQvPb9defKY6w8YW8rprDhKiTqkmexp1P8ioUznY', 'posture': 'public'}}


### THIS WILL FAIL

In [ ]:
schema_id = 'MGgoJXWbeupKsaHDa7s4fW:2:testabc:0.0.1'
response = await agent_controller.definitions.write_cred_def(schema_id)

## 3. Generate a new DID

Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verification key for that DID. A representation of it's public key. 

In [5]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

New DID {'did': 'Jqi9JNk9Z637azKekx51V4', 'verkey': 'AixCmDcPfWvZsuMpV2swhvukSYn2e9JbQjXMq4C2GD1H', 'posture': 'wallet_only'}


## 4. Write DID to Sovrin Stagingnet

Anoyone can write a DID to the Sovrin StagingNet, it is a permissionless ledger. 

Visit [Sovrin Selfserve Portal](https://selfserve.sovrin.org) for more information. We have provided an automated process to write DIDs to Stagingnet in the step below.

In [6]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())
print(r.status_code)

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "Jqi9JNk9Z637azKekx51V4": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by Jqi9JNk9Z637azKekx51V4 to the ledger with role ENDORSER"}}'}
200


## 5. Accepting the Transaction Author Agreement (TAA)

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [7]:
# This will not work until you have accepted the TAA
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

ClientResponseError: 400, message="Ledger rejected transaction request: client request invalid: InvalidClientTaaAcceptanceError('Txn Author Agreement acceptance is required for ledger with id 1',).", url=URL('http://issuer-agent:8021/wallet/did/public?did=Jqi9JNk9Z637azKekx51V4')

Error during POST /wallet/did/public: 400, message="Ledger rejected transaction request: client request invalid: InvalidClientTaaAcceptanceError('Txn Author Agreement acceptance is required for ledger with id 1',).", url=URL('http://issuer-agent:8021/wallet/did/public?did=Jqi9JNk9Z637azKekx51V4')


In [8]:
response = await agent_controller.ledger.get_taa()
TAA = response['result']['taa_record']
TAA['mechanism'] = "service_agreement"
print(TAA)

{'text': '\ufeff# Transaction Author Agreement V2\nhttps://sovrin.org/\n\n\n## Summary:\n\n\nThis summary is provided to help you understand your obligations when writing to\nthe Sovrin Ledger Networks-it does not have any legal effect or replace the full\nlegal text of the agreement provided below it.\n\n\n- This agreement grants you permission to write data to the Sovrin Ledger\n  Networks under certain terms and conditions.\n\n\n- You represent and warrant that the data you are writing does not violate any\n  applicable laws or infringe the rights of any other party.\n\n\n- You understand the data you are writing is public and permanent and there can\n  be no guarantee of erasure. This includes public keys and payment addresses.\n\n\n- If it is determined that the data you wrote violated this agreement, the\n  operators of the network can take steps to block it from public access.\n\n\n- The Sovrin Foundation makes no promises about the reliability or correctness\n  of the data bein

In [9]:
response = await agent_controller.ledger.accept_taa(TAA)
## Will return {} if successful
print(response)

{}


## 6. Set public DID

Now you are able to assign the DID written to the ledger as public.

In [10]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

{'result': {'did': 'Jqi9JNk9Z637azKekx51V4', 'verkey': 'AixCmDcPfWvZsuMpV2swhvukSYn2e9JbQjXMq4C2GD1H', 'posture': 'public'}}


## 7. Get public DID

In [11]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_nym = response['result']['did']
print('\nIssuer public DID:',issuer_nym)

{'result': {'did': 'Jqi9JNk9Z637azKekx51V4', 'verkey': 'AixCmDcPfWvZsuMpV2swhvukSYn2e9JbQjXMq4C2GD1H', 'posture': 'public'}}

Issuer public DID: Jqi9JNk9Z637azKekx51V4


## 8. Fetch verkey for public DID

Additionally, we can verify that this DID does actually resolve to the public key material on the ledger.

In [12]:
issuer_verkey = await agent_controller.ledger.get_did_verkey(issuer_nym)
print(issuer_verkey)

{'verkey': 'AixCmDcPfWvZsuMpV2swhvukSYn2e9JbQjXMq4C2GD1H'}


## 9. Get public DID endpoint

As well as providing a publically accessible endpoint to contact the DID controller through the agent framework.

In [13]:
issuer_endpoint = await agent_controller.ledger.get_did_endpoint(issuer_nym)
print(issuer_endpoint)

{'endpoint': 'https://1ae2ec7766b1.ngrok.io'}


## 10. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)

# Proceed to Part 3 on [Issuer Notebook](http://localhost:8888/notebooks/Part%203%20-%20Issue%20Credential.ipynb)

Here you will be issued with a credential into your mobile SSI wallet that will be verified in part 4.